# SCENIC+: Pipeline

# SCENIC+: Step-by-step

## 1. Set-up

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import os
_stderr = sys.stderr
null = open(os.devnull,'wb')

In [2]:
!ls -lh /cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/encode/fragments

total 33K
drwxr-xr-x 3 aklie carter-users   1 Jan  5 10:03 ENCFF035SPT
drwxr-xr-x 3 aklie carter-users   1 Jan  5 10:03 ENCFF042ZJI
drwxr-xr-x 3 aklie carter-users   1 Jan  5 10:04 ENCFF101BLM
drwxr-xr-x 3 aklie carter-users   1 Jan  5 10:04 ENCFF119IVK
drwxr-xr-x 3 aklie carter-users   1 Jan  5 10:04 ENCFF176LJV
drwxr-xr-x 3 aklie carter-users   1 Jan  5 10:04 ENCFF187VMN
drwxr-xr-x 3 aklie carter-users   1 Jan  5 10:04 ENCFF622EUO
drwxr-xr-x 3 aklie carter-users   1 Jan  5 10:05 ENCFF683IBE
-rw-r--r-- 1 aklie carter-users 29K Jan  5 09:55 metadata.tsv


In [3]:
!ls -lh /cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/preprocess/snrna/filtered.h5ad

-rw-r--r-- 1 aklie carter-users 801M Jan 13 09:46 /cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/preprocess/snrna/filtered.h5ad


## 2. scRNA-seq analysis

In [4]:
work_dir = 'mouse_adrenal'

In [5]:
import scanpy as sc
#set some figure parameters for nice display inside jupyternotebooks.
%matplotlib inline
sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(5, 5), facecolor='white')

#make a directory for to store the processed scRNA-seq data.
if not os.path.exists(os.path.join(work_dir, 'scRNA')):
    os.makedirs(os.path.join(work_dir, 'scRNA'))

In [6]:
adata = sc.read_h5ad(os.path.join(work_dir, 'data/adata.h5ad'))
adata.var_names_make_unique()
adata

AnnData object with n_obs × n_vars = 79209 × 47721
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'cellID', 'doublet_scores', 'doublets', 'library_accession', 'technology', 'species', 'tissue', 'sex', 'timepoint', 'rep', 'sample', 'depth1', 'depth2', 'experiment', 'experiment_batch', 'integration_batch', 'run_number', 'experiment_accession', 'file_accession', 'lower_nCount_RNA', 'upper_nCount_RNA', 'lower_nFeature_RNA', 'upper_doublet_scores', 'upper_percent.mt', 'percent.mt', 'percent.ribo', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.1.6', 'seurat_clusters', 'S.Score', 'G2M.Score', 'Phase', 'subtypes', 'celltypes', 'gen_celltype', 'Cortex_membership_score', 'Endothelial_membership_score', 'Adipocytes_membership_score', 'Myeloid_membership_score', 'Sox10._membership_score', 'Fibroblast_membership_score', 'Medulla_membership_score', 'Stromal_membership_score', 'Smooth_muscle_membership_score', 'Capsule_membership_score', 'Hepatocyte_membership_score', 'Myonuclei_membership_

## 3. scATAC-seq analysis

In [7]:
import pycisTopic
#set some figure parameters for nice display inside jupyternotebooks.
%matplotlib inline

#make a directory for to store the processed scRNA-seq data.
if not os.path.exists(os.path.join(work_dir, 'scATAC')):
    os.makedirs(os.path.join(work_dir, 'scATAC'))
tmp_dir = '/cellar/users/aklie/tmp/'

In [8]:
import glob as glob
frag_files = glob.glob("/cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/encode/fragments/*/*/*/*/*/*.tsv.gz")
fragments_dict = dict(zip([file.split("/")[-6] for file in frag_files], frag_files))

In [9]:
import pandas as pd
snatac_meta = pd.read_csv("/cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/encode/enc4_mouse_snatac_metadata.tsv", sep="\t")

In [10]:
cell_data = adata.obs
del(adata)

In [11]:
cell_data = cell_data[cell_data["technology"] == "10x"]
acc_mp = snatac_meta.set_index("rna_library_accession")["file_accession"]
cell_data["sample_id"] = cell_data["library_accession"].map(acc_mp).values
cell_data['celltype'] = cell_data['celltypes'].astype(str) # set data type of the celltype column to str, otherwise the export_pseudobulk function will complain.
cell_data["sample_id"].value_counts(dropna=False)

/tmp/ipykernel_1990380/2069487882.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_data["sample_id"] = cell_data["library_accession"].map(acc_mp).values
/tmp/ipykernel_1990380/2069487882.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_data['celltype'] = cell_data['celltypes'].astype(str) # set data type of the celltype column to str, otherwise the export_pseudobulk function will complain.


ENCFF683IBE    5270
ENCFF176LJV    5163
ENCFF035SPT    4584
ENCFF622EUO    4507
ENCFF101BLM    4324
ENCFF119IVK    3989
ENCFF042ZJI    3987
ENCFF187VMN    3172
Name: sample_id, dtype: int64

In [12]:
cell_data["rna_bc"] = [row[0] for row in cell_data["cellID"].str.split(".")]

/tmp/ipykernel_1990380/2848274305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_data["rna_bc"] = [row[0] for row in cell_data["cellID"].str.split(".")]


In [13]:
rna_bcs = pd.read_csv("/cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/encode/gene_exp_737K-arc-v1.txt", header=None)[0].values
atac_bcs = pd.read_csv("/cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/encode/atac_737K-arc-v1.txt", header=None)[0].values
bcs = pd.DataFrame(data={"rna_bcs": rna_bcs, "atac_bcs": atac_bcs})

In [14]:
COMPLEMENT_DNA = {"A": "T", "C": "G", "G": "C", "T": "A"}
bcs["atac_bcs_rc"] = ["".join(COMPLEMENT_DNA.get(base, base) for base in reversed(bc)) for bc in bcs["atac_bcs"]]

In [15]:
bc_map = bcs.set_index("rna_bcs")["atac_bcs_rc"]

In [16]:
cell_data["atac_bc"] = cell_data["rna_bc"].map(bc_map)

/tmp/ipykernel_1990380/2987911187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_data["atac_bc"] = cell_data["rna_bc"].map(bc_map)


In [17]:
cell_data["atac_bc_sample"] = cell_data["atac_bc"] + "-" + cell_data["sample_id"]

/tmp/ipykernel_1990380/370147507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_data["atac_bc_sample"] = cell_data["atac_bc"] + "-" + cell_data["sample_id"]


In [18]:
import pyranges as pr
import requests
target_url='https://hgdownload.cse.ucsc.edu/goldenpath/mm10/bigZips/mm10.chrom.sizes'
chromsizes=pd.read_csv(target_url, sep='\t', header=None)
chromsizes.columns=['Chromosome', 'End']
chromsizes['Start']=[0]*chromsizes.shape[0]
chromsizes=chromsizes.loc[:,['Chromosome', 'Start', 'End']]
# Exceptionally in this case, to agree with CellRangerARC annotations
#chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].replace('v', '.') for x in range(len(chromsizes['Chromosome']))]
#chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].split('_')[1] if len(chromsizes['Chromosome'][x].split('_')) > 1 else chromsizes['Chromosome'][x] for x in range(len(chromsizes['Chromosome']))]
chromsizes=pr.PyRanges(chromsizes)

In [19]:
chromsizes

,Chromosome,Start,End
0,chr1,0,195471971
1,chr1_GL456210_random,0,169725
2,chr1_GL456211_random,0,241735
3,chr1_GL456212_random,0,153618
4,chr1_GL456213_random,0,39340
...,...,...,...
61,chrY,0,91744698
62,chrY_JH584300_random,0,182347
63,chrY_JH584301_random,0,259875
64,chrY_JH584302_random,0,155838


In [ ]:
test_fragments_dict = {"ENCFF187VMN": fragments_dict["ENCFF187VMN"]}
test_cell_data = cell_data[cell_data["sample_id"] == "ENCFF187VMN"]
test_cell_data["barcode"] = test_cell_data["atac_bc"]
test_cell_data = test_cell_data.set_index("atac_bc")
test_fragments = pd.read_csv(test_fragments_dict["ENCFF187VMN"], sep="\t", header=None)

In [ ]:
test_cell_data["barcode"].isin(test_fragments[3]).sum()

In [ ]:
import gc
import logging
import os
import re
import subprocess
import sys
from typing import Dict, List, Optional, Union
import numpy as np
import pandas as pd
import pyBigWig
import pyranges as pr

def export_pseudobulk_one_sample(
    cell_data: pd.DataFrame,
    group: str,
    fragments_df_dict: Dict[str, pd.DataFrame],
    chromsizes: pr.PyRanges,
    bigwig_path: str,
    bed_path: str,
    sample_id_col: Optional[str] = "sample_id",
    normalize_bigwig: Optional[bool] = True,
    remove_duplicates: Optional[bool] = True,
    split_pattern: Optional[str] = "___",
):
    """
    Create pseudobulk as bed and bigwig from single cell fragments file given a barcode annotation and a group.
    Parameters
    ---------
    cell_data: pd.DataFrame
            A cell metadata :class:`pd.Dataframe` containing barcodes, their annotation and their sample of origin.
    group: str
            A character string indicating the group for which pseudobulks will be created.
    fragments_df_dict: dict
            A dictionary containing data frames as values with 'Chromosome', 'Start', 'End', 'Name', and 'Score' as columns; and sample label
            as keys. 'Score' indicates the number of times that a fragments is found assigned to that barcode.
    chromsizes: pr.PyRanges
            A :class:`pr.PyRanges` containing size of each column, containing 'Chromosome', 'Start' and 'End' columns.
    bigwig_path: str
            Path to folder where the bigwig file will be saved.
    bed_path: str
            Path to folder where the fragments bed file will be saved.
    sample_id_col: str, optional
            Name of the column containing the sample name per barcode in the input :class:`CistopicObject.cell_data` or class:`pd.DataFrame`. Default: 'sample_id'.
    normalize_bigwig: bool, optional
            Whether bigwig files should be CPM normalized. Default: True.
    remove_duplicates: bool, optional
            Whether duplicates should be removed before converting the data to bigwig.
    split_pattern: str
            Pattern to split cell barcode from sample id. Default: ___ .
    """
    # Create logger
    level = logging.INFO
    log_format = "%(asctime)s %(name)-12s %(levelname)-8s %(message)s"
    handlers = [logging.StreamHandler(stream=sys.stdout)]
    logging.basicConfig(level=level, format=log_format, handlers=handlers)
    log = logging.getLogger("cisTopic")

    log.info("Creating pseudobulk for " + str(group))
    group_fragments_list = []
    group_fragments_dict = {}
    for sample_id in fragments_df_dict:
        sample_data = cell_data[cell_data.loc[:, sample_id_col].isin([sample_id])]
        if "barcode" in sample_data:
            sample_data.index = sample_data["barcode"].tolist()
        else:
            sample_data.index = prepare_tag_cells(
                sample_data.index.tolist(), split_pattern
            )
        group_var = sample_data.iloc[:, 0]
        barcodes = group_var[group_var.isin([group])].index.tolist()
        print(barcodes)
        fragments_df = fragments_df_dict[sample_id]
        group_fragments = fragments_df.loc[fragments_df["Name"].isin(barcodes)]
        if len(fragments_df_dict) > 1:
            group_fragments_dict[sample_id] = group_fragments

    if len(fragments_df_dict) > 1:
        group_fragments_list = [
            group_fragments_dict[list(group_fragments_dict.keys())[x]]
            for x in range(len(fragments_df_dict))
        ]
        group_fragments = group_fragments_list[0].append(group_fragments_list[1:])

    del group_fragments_dict
    del group_fragments_list
    del fragments_df
    gc.collect()

    group_pr = pr.PyRanges(group_fragments)
    if isinstance(bigwig_path, str):
        bigwig_path_group = os.path.join(bigwig_path, str(group) + ".bw")
        if remove_duplicates:
            group_pr.to_bigwig(
                path=bigwig_path_group,
                chromosome_sizes=chromsizes,
                rpm=normalize_bigwig,
            )
        else:
            group_pr.to_bigwig(
                path=bigwig_path_group,
                chromosome_sizes=chromsizes,
                rpm=normalize_bigwig,
                value_col="Score",
            )
    if isinstance(bed_path, str):
        print("here")
        print(group_pr)
        bed_path_group = os.path.join(bed_path, str(group) + ".bed.gz")
        group_pr.to_bed(
            path=bed_path_group, keep=False, compression="infer", chain=False
        )

    log.info(str(group) + " done!")

In [ ]:
from pycisTopic.cistopic_class import *
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk_ray
from pycisTopic.utils import read_fragments_from_file, prepare_tag_cells
def export_pseudobulk(
    input_data: Union["CistopicObject", pd.DataFrame, Dict[str, pd.DataFrame]],
    variable: str,
    chromsizes: Union[pd.DataFrame, pr.PyRanges],
    bed_path: str,
    bigwig_path: str,
    path_to_fragments: Optional[Dict[str, str]] = None,
    sample_id_col: Optional[str] = "sample_id",
    n_cpu: Optional[int] = 1,
    normalize_bigwig: Optional[bool] = True,
    remove_duplicates: Optional[bool] = True,
    split_pattern: Optional[str] = "___",
    use_polars: Optional[bool] = True,
    **kwargs
):
    """
    Create pseudobulks as bed and bigwig from single cell fragments file given a barcode annotation.
    Parameters
    ---------
    input_data: CistopicObject or pd.DataFrame
            A :class:`CistopicObject` containing the specified `variable` as a column in :class:`CistopicObject.cell_data` or a cell metadata
            :class:`pd.DataFrame` containing barcode as rows, containing the specified `variable` as a column (additional columns are
            possible) and a `sample_id` column. Index names must contain the BARCODE (e.g. ATGTCGTC-1), additional tags are possible separating with -
            (e.g. ATGCTGTGCG-1-Sample_1). The levels in the sample_id column must agree with the keys in the path_to_fragments dictionary.
            Alternatively, if the cell metadata contains a column named barcode it will be used instead of the index names.
    variable: str
            A character string indicating the column that will be used to create the different group pseudobulk. It must be included in
            the cell metadata provided as input_data.
    chromsizes: pd.DataFrame or pr.PyRanges
            A data frame or :class:`pr.PyRanges` containing size of each chromosome, containing 'Chromosome', 'Start' and 'End' columns.
    bed_path: str
            Path to folder where the fragments bed files per group will be saved. If None, files will not be generated.
    bigwig_path: str
            Path to folder where the bigwig files per group will be saved. If None, files will not be generated.
    path_to_fragments: str or dict, optional
            A dictionary of character strings, with sample name as names indicating the path to the fragments file/s from which pseudobulk profiles have to
            be created. If a :class:`CistopicObject` is provided as input it will be ignored, but if a cell metadata :class:`pd.DataFrame` is provided it
            is necessary to provide it. The keys of the dictionary need to match with the sample_id tag added to the index names of the input data frame.
    sample_id_col: str, optional
            Name of the column containing the sample name per barcode in the input :class:`CistopicObject.cell_data` or class:`pd.DataFrame`. Default: 'sample_id'.
    n_cpu: int, optional
            Number of cores to use. Default: 1.
    normalize_bigwig: bool, optional
            Whether bigwig files should be CPM normalized. Default: True.
    remove_duplicates: bool, optional
            Whether duplicates should be removed before converting the data to bigwig.
    split_pattern: str, optional
            Pattern to split cell barcode from sample id. Default: ___ .
    use_polars: bool, optional
            Whether to use polars to read fragments files. Default: True.
    **kwargs
            Additional parameters for ray.init()
    Return
    ------
    dict
            A dictionary containing the paths to the newly created bed fragments files per group a dictionary containing the paths to the
            newly created bigwig files per group.
    """
    # Create logger
    level = logging.INFO
    log_format = "%(asctime)s %(name)-12s %(levelname)-8s %(message)s"
    handlers = [logging.StreamHandler(stream=sys.stdout)]
    logging.basicConfig(level=level, format=log_format, handlers=handlers)
    log = logging.getLogger("cisTopic")

    # Get fragments file
    if isinstance(input_data, CistopicObject):
        path_to_fragments = input_data.path_to_fragments
        if path_to_fragments is None:
            log.error("No path_to_fragments in this cisTopic object.")
        cell_data = input_data.cell_data
    elif isinstance(input_data, pd.DataFrame):
        if path_to_fragments is None:
            log.error("Please, provide path_to_fragments.")
        cell_data = input_data
    # Check for sample_id column
    try:
        sample_ids = list(set(cell_data[sample_id_col]))
    except ValueError:
        print(
            'Please, include a sample identification column (e.g. "sample_id") in your cell metadata!'
        )

    # Get fragments
    #fragments_df_dict = {}
    """
    for sample_id in path_to_fragments.keys():
        if sample_id not in sample_ids:
            log.info(
                "The following path_to_fragments entry is not found in the cell metadata sample_id_col: ",
                sample_id,
                ". It will be ignored.",
            )
        else:
            log.info("Reading fragments from " + path_to_fragments[sample_id])
            fragments_df = read_fragments_from_file(path_to_fragments[sample_id], use_polars=use_polars).df
            # Convert to int32 for memory efficiency
            fragments_df.Start = np.int32(fragments_df.Start)
            fragments_df.End = np.int32(fragments_df.End)
            if "Score" in fragments_df:
                fragments_df.Score = np.int32(fragments_df.Score)
            if "barcode" in cell_data:
                fragments_df = fragments_df.loc[
                    fragments_df["Name"].isin(cell_data["barcode"].tolist())
                ]
            else:
                fragments_df = fragments_df.loc[
                    fragments_df["Name"].isin(
                        prepare_tag_cells(cell_data.index.tolist(), split_pattern)
                    )
                ]
            fragments_df_dict[sample_id] = fragments_df
    """
    
    # Set groups
    if "barcode" in cell_data:
        cell_data = cell_data.loc[:, [variable, sample_id_col, "barcode"]]
    else:
        cell_data = cell_data.loc[:, [variable, sample_id_col]]
    cell_data[variable] = cell_data[variable].replace(" ", "", regex=True)
    cell_data[variable] = cell_data[variable].replace("[^A-Za-z0-9]+", "_", regex=True)
    groups = sorted(list(set(cell_data[variable])))
    groups = groups[:2]
    print(groups)
    # Check chromosome sizes
    if isinstance(chromsizes, pd.DataFrame):
        chromsizes = chromsizes.loc[:, ["Chromosome", "Start", "End"]]
        chromsizes = pr.PyRanges(chromsizes)
    # Check that output dir exist and generate output paths
    if isinstance(bed_path, str):
        if not os.path.exists(bed_path):
            os.makedirs(bed_path)
        bed_paths = {
            group: os.path.join(bed_path, str(group) + ".bed.gz") for group in groups
        }
    else:
        bed_paths = {}
    if isinstance(bigwig_path, str):
        if not os.path.exists(bigwig_path):
            os.makedirs(bigwig_path)
        bw_paths = {
            group: os.path.join(bigwig_path, str(group) + ".bw") for group in groups
        }
    else:
        bw_paths = {}
    # Create pseudobulks
    print(cell_data)
    print(groups)
    print(fragments_df_dict)
    print(chromsizes)
    print(bigwig_path, bed_path)
    print(sample_id_col)
    if n_cpu > 1:
        ray.init(num_cpus=n_cpu, **kwargs)
        ray_handle = ray.wait(
            [
                export_pseudobulk_ray.remote(
                    cell_data,
                    group,
                    fragments_df_dict,
                    chromsizes,
                    bigwig_path,
                    bed_path,
                    sample_id_col,
                    normalize_bigwig,
                    remove_duplicates,
                    split_pattern,
                )
                for group in groups
            ],
            num_returns=len(groups),
        )
        ray.shutdown()
    else:
        [
            export_pseudobulk_one_sample(
                cell_data,
                group,
                fragments_df_dict,
                chromsizes,
                bigwig_path,
                bed_path,
                sample_id_col,
                normalize_bigwig,
                remove_duplicates,
                split_pattern,
            )
            for group in groups
        ]

    return bw_paths, bed_paths

In [ ]:
bw_paths, bed_paths = export_pseudobulk(
    input_data = test_cell_data,
    variable = 'celltypes', # variable by which to generate pseubulk profiles, in this case we want pseudobulks per celltype
    sample_id_col = 'sample_id',
    chromsizes = chromsizes,
    bed_path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bed_files/'),  # specify where pseudobulk_bed_files should be stored
    bigwig_path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bw_files/'), # specify where pseudobulk_bw_files should be stored
    path_to_fragments = test_fragments_dict, # location of fragment fiels
    n_cpu = 1, # specify the number of cores to use, we use ray for multi processing
    normalize_bigwig = True,
    remove_duplicates = True,
    _temp_dir = os.path.join(tmp_dir, 'ray_spill'),
    split_pattern = '-'
)

In [ ]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
bw_paths, bed_paths = export_pseudobulk(
    input_data = cell_data,
    variable = 'celltype', # variable by which to generate pseubulk profiles, in this case we want pseudobulks per celltype
    sample_id_col = 'sample_id',
    chromsizes = chromsizes,
    bed_path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bed_files/'),  # specify where pseudobulk_bed_files should be stored
    bigwig_path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bw_files/'), # specify where pseudobulk_bw_files should be stored
    path_to_fragments = fragments_dict, # location of fragment fiels
    n_cpu = 12, # specify the number of cores to use, we use ray for multi processing
    normalize_bigwig = True,
    remove_duplicates = True,
    _temp_dir = os.path.join(tmp_dir, 'ray_spill'),
    split_pattern = '-'
)

In [ ]:
import logging

In [ ]:
path_to_fragments = test_fragments_dict
sample_id_col = "sample_id"
variable = "celltypes"
sample_ids = list(set(cell_data[sample_id_col]))
use_polars = True
level = logging.INFO
log_format = "%(asctime)s %(name)-12s %(levelname)-8s %(message)s"
handlers = [logging.StreamHandler(stream=sys.stdout)]
logging.basicConfig(level=level, format=log_format, handlers=handlers)
log = logging.getLogger("cisTopic")
cell_data = test_cell_data
bed_path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bed_files/')
bigwig_path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bw_files/'),

In [ ]:
from pycisTopic.utils import read_fragments_from_file, prepare_tag_cells

In [ ]:
import numpy as np

In [ ]:
fragments_df_dict = {}
for sample_id in path_to_fragments.keys():
    if sample_id not in sample_ids:
        log.info(
            "The following path_to_fragments entry is not found in the cell metadata sample_id_col: ",
            sample_id,
            ". It will be ignored.",
        )
    else:
        log.info("Reading fragments from " + path_to_fragments[sample_id])

        fragments_df = read_fragments_from_file(path_to_fragments[sample_id], use_polars=use_polars).df
        # Convert to int32 for memory efficiency
        fragments_df.Start = np.int32(fragments_df.Start)
        fragments_df.End = np.int32(fragments_df.End)
        if "Score" in fragments_df:
            fragments_df.Score = np.int32(fragments_df.Score)
        if "barcode" in cell_data:
            fragments_df = fragments_df.loc[
                fragments_df["Name"].isin(cell_data["barcode"].tolist())
            ]
        else:
            fragments_df = fragments_df.loc[
                fragments_df["Name"].isin(
                    prepare_tag_cells(cell_data.index.tolist(), split_pattern)
                )
            ]
        fragments_df_dict[sample_id] = fragments_df

In [ ]:
if "barcode" in cell_data:
    cell_data = cell_data.loc[:, [variable, sample_id_col, "barcode"]]
else:
    cell_data = cell_data.loc[:, [variable, sample_id_col]]
cell_data[variable] = cell_data[variable].replace(" ", "", regex=True)
cell_data[variable] = cell_data[variable].replace("[^A-Za-z0-9]+", "_", regex=True)
groups = sorted(list(set(cell_data[variable])))

In [ ]:
cell_data

In [ ]:
group = groups[0]

In [ ]:
level = logging.INFO
log_format = "%(asctime)s %(name)-12s %(levelname)-8s %(message)s"
handlers = [logging.StreamHandler(stream=sys.stdout)]
logging.basicConfig(level=level, format=log_format, handlers=handlers)
log = logging.getLogger("cisTopic")

log.info("Creating pseudobulk for " + str(group))
group_fragments_list = []
group_fragments_dict = {}

In [ ]:
log.info("Creating pseudobulk for " + str(group))
group_fragments_list = []
group_fragments_dict = {}
for sample_id in fragments_df_dict:
    print(sample_id)
    sample_data = cell_data[cell_data.loc[:, sample_id_col].isin([sample_id])]
    if "barcode" in sample_data:
        sample_data.index = sample_data["barcode"].tolist()
    else:
        sample_data.index = prepare_tag_cells(
            sample_data.index.tolist(), split_pattern
        )
    group_var = sample_data.iloc[:, 0]
    barcodes = group_var[group_var.isin([group])].index.tolist()
    fragments_df = fragments_df_dict[sample_id]
    group_fragments = fragments_df.loc[fragments_df["Name"].isin(barcodes)]
    if len(fragments_df_dict) > 1:
        group_fragments_dict[sample_id] = group_fragments

In [ ]:
sample_data

In [ ]:
sample_data = cell_data[cell_data.loc[:, sample_id_col].isin([sample_id])]

In [ ]:
if len(fragments_df_dict) > 1:
        group_fragments_list = [
            group_fragments_dict[list(group_fragments_dict.keys())[x]]
            for x in range(len(fragments_df_dict))
        ]
        group_fragments = group_fragments_list[0].append(group_fragments_list[1:])

In [ ]:
del group_fragments_dict
del group_fragments_list
del fragments_df
gc.collect()

group_pr = pr.PyRanges(group_fragments)

In [ ]:
group_pr

In [ ]:
if isinstance(bigwig_path, str):
    bigwig_path_group = os.path.join(bigwig_path, str(group) + ".bw")
    if remove_duplicates:
        group_pr.to_bigwig(
            path=bigwig_path_group,
            chromosome_sizes=chromsizes,
            rpm=normalize_bigwig,
        )
    else:
        group_pr.to_bigwig(
            path=bigwig_path_group,
            chromosome_sizes=chromsizes,
            rpm=normalize_bigwig,
            value_col="Score",
        )
if isinstance(bed_path, str):
    print("here")
    print(group_pr)
    bed_path_group = os.path.join(bed_path, str(group) + ".bed.gz")
    group_pr.to_bed(
        path=bed_path_group, keep=False, compression="infer", chain=False
    )

log.info(str(group) + " done!")

In [ ]:
import gc
import logging
import os
import re
import subprocess
import sys
from typing import Dict, List, Optional, Union
import numpy as np
import pandas as pd
import pyBigWig
import pyranges as pr

def export_pseudobulk_one_sample(
    cell_data: pd.DataFrame,
    group: str,
    fragments_df_dict: Dict[str, pd.DataFrame],
    chromsizes: pr.PyRanges,
    bigwig_path: str,
    bed_path: str,
    sample_id_col: Optional[str] = "sample_id",
    normalize_bigwig: Optional[bool] = True,
    remove_duplicates: Optional[bool] = True,
    split_pattern: Optional[str] = "___",
):
    """
    Create pseudobulk as bed and bigwig from single cell fragments file given a barcode annotation and a group.
    Parameters
    ---------
    cell_data: pd.DataFrame
            A cell metadata :class:`pd.Dataframe` containing barcodes, their annotation and their sample of origin.
    group: str
            A character string indicating the group for which pseudobulks will be created.
    fragments_df_dict: dict
            A dictionary containing data frames as values with 'Chromosome', 'Start', 'End', 'Name', and 'Score' as columns; and sample label
            as keys. 'Score' indicates the number of times that a fragments is found assigned to that barcode.
    chromsizes: pr.PyRanges
            A :class:`pr.PyRanges` containing size of each column, containing 'Chromosome', 'Start' and 'End' columns.
    bigwig_path: str
            Path to folder where the bigwig file will be saved.
    bed_path: str
            Path to folder where the fragments bed file will be saved.
    sample_id_col: str, optional
            Name of the column containing the sample name per barcode in the input :class:`CistopicObject.cell_data` or class:`pd.DataFrame`. Default: 'sample_id'.
    normalize_bigwig: bool, optional
            Whether bigwig files should be CPM normalized. Default: True.
    remove_duplicates: bool, optional
            Whether duplicates should be removed before converting the data to bigwig.
    split_pattern: str
            Pattern to split cell barcode from sample id. Default: ___ .
    """
    # Create logger
    level = logging.INFO
    log_format = "%(asctime)s %(name)-12s %(levelname)-8s %(message)s"
    handlers = [logging.StreamHandler(stream=sys.stdout)]
    logging.basicConfig(level=level, format=log_format, handlers=handlers)
    log = logging.getLogger("cisTopic")

    log.info("Creating pseudobulk for " + str(group))
    group_fragments_list = []
    group_fragments_dict = {}
    for sample_id in fragments_df_dict:
        sample_data = cell_data[cell_data.loc[:, sample_id_col].isin([sample_id])]
        if "barcode" in sample_data:
            sample_data.index = sample_data["barcode"].tolist()
        else:
            sample_data.index = prepare_tag_cells(
                sample_data.index.tolist(), split_pattern
            )
        group_var = sample_data.iloc[:, 0]
        barcodes = group_var[group_var.isin([group])].index.tolist()
        fragments_df = fragments_df_dict[sample_id]
        group_fragments = fragments_df.loc[fragments_df["Name"].isin(barcodes)]
        if len(fragments_df_dict) > 1:
            group_fragments_dict[sample_id] = group_fragments

    if len(fragments_df_dict) > 1:
        group_fragments_list = [
            group_fragments_dict[list(group_fragments_dict.keys())[x]]
            for x in range(len(fragments_df_dict))
        ]
        group_fragments = group_fragments_list[0].append(group_fragments_list[1:])

    del group_fragments_dict
    del group_fragments_list
    del fragments_df
    gc.collect()

    group_pr = pr.PyRanges(group_fragments)
    if isinstance(bigwig_path, str):
        bigwig_path_group = os.path.join(bigwig_path, str(group) + ".bw")
        if remove_duplicates:
            group_pr.to_bigwig(
                path=bigwig_path_group,
                chromosome_sizes=chromsizes,
                rpm=normalize_bigwig,
            )
        else:
            group_pr.to_bigwig(
                path=bigwig_path_group,
                chromosome_sizes=chromsizes,
                rpm=normalize_bigwig,
                value_col="Score",
            )
    if isinstance(bed_path, str):
        print("here")
        print(group_pr)
        bed_path_group = os.path.join(bed_path, str(group) + ".bed.gz")
        group_pr.to_bed(
            path=bed_path_group, keep=False, compression="infer", chain=False
        )

    log.info(str(group) + " done!")

In [ ]:
sample_data

In [ ]:
fragments_df_dict["ENCFF187VMN"]

In [ ]:
export_pseudobulk_one_sample(
    cell_data,
    groups[2],
    fragments_df_dict,
    chromsizes,
    bigwig_path,
    bed_path,
    sample_id_col,
    True,
    True,
    "-"
)

In [ ]:
test = [
export_pseudobulk_one_sample(
    cell_data,
    group,
    fragments_df_dict,
    chromsizes,
    bigwig_path,
    bed_path,
    sample_id_col,
    True,
    True,
    "-",
)
for group in groups
]

In [ ]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
bw_paths, bed_paths = export_pseudobulk(
    input_data = test_cell_data,
    variable = 'celltypes', # variable by which to generate pseubulk profiles, in this case we want pseudobulks per celltype
    sample_id_col = 'sample_id',
    chromsizes = chromsizes,
    bed_path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bed_files/'),  # specify where pseudobulk_bed_files should be stored
    bigwig_path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bw_files/'), # specify where pseudobulk_bw_files should be stored
    path_to_fragments = test_fragments_dict, # location of fragment fiels
    n_cpu = 1, # specify the number of cores to use, we use ray for multi processing
    normalize_bigwig = True,
    remove_duplicates = True,
    _temp_dir = os.path.join(tmp_dir, 'ray_spill'),
    split_pattern = '-'
)

In [ ]:
len(cell_data)

In [ ]:
cell_data["sample_id"].isin(fragments_dict.keys()).sum()

In [ ]:
cell_data["celltype"].value_counts(dropna=False)

In [20]:
cell_data["barcode"] = cell_data["atac_bc"]
cell_data = cell_data.set_index("atac_bc")

/tmp/ipykernel_1990380/2069362678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_data["barcode"] = cell_data["atac_bc"]


In [26]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
bw_paths, bed_paths = export_pseudobulk(
    input_data = cell_data,
    variable = 'celltype', # variable by which to generate pseubulk profiles, in this case we want pseudobulks per celltype
    sample_id_col = 'sample_id',
    chromsizes = chromsizes,
    bed_path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bed_files/'),  # specify where pseudobulk_bed_files should be stored
    bigwig_path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bw_files/'), # specify where pseudobulk_bw_files should be stored
    path_to_fragments = fragments_dict, # location of fragment fiels
    n_cpu = 1, # specify the number of cores to use, we use ray for multi processing
    normalize_bigwig = True,
    remove_duplicates = True,
    _temp_dir = os.path.join(tmp_dir, 'ray_spill'),
    split_pattern = '-'
)

2023-01-14 22:54:59,585 cisTopic     INFO     Reading fragments from /cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/encode/fragments/ENCFF187VMN/encode_scatac_dcc_2/results/ENCSR525WPH-1/fragments/fragments.tsv.gz
2023-01-14 22:56:44,138 cisTopic     INFO     Reading fragments from /cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/encode/fragments/ENCFF035SPT/encode_scatac_dcc_2/results/ENCSR713FPX-1/fragments/fragments.tsv.gz
2023-01-14 22:58:25,383 cisTopic     INFO     Reading fragments from /cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/encode/fragments/ENCFF622EUO/encode_scatac_dcc_2/results/ENCSR858YSB-1/fragments/fragments.tsv.gz
2023-01-14 23:00:24,434 cisTopic     INFO     Reading fragments from /cellar/users/aklie/data/igvf/topic_grn_links/mouse_adrenal/encode/fragments/ENCFF119IVK/encode_scatac_dcc_2/results/ENCSR400PXQ-1/fragments/fragments.tsv.gz
2023-01-14 23:01:55,753 cisTopic     INFO     Reading fragments from /cellar/users/aklie/dat

In [27]:
import pickle
pickle.dump(bed_paths,
            open(os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bed_files/bed_paths.pkl'), 'wb'))
pickle.dump(bw_paths,
           open(os.path.join(work_dir, 'scATAC/consensus_peak_calling/pseudobulk_bed_files/bw_paths.pkl'), 'wb'))

In [ ]:
from pycisTopic.pseudobulk_peak_calling import peak_calling
macs_path='/cellar/users/aklie/opt/miniconda3/envs/scenicplus/bin/macs2'
# Run peak calling
narrow_peaks_dict = peak_calling(macs_path,
                                 bed_paths,
                                 os.path.join(work_dir, 'scATAC/consensus_peak_calling/MACS/'),
                                 genome_size='hs',
                                 n_cpu=12,
                                 input_format='BEDPE',
                                 shift=73,
                                 ext_size=146,
                                 keep_dup = 'all',
                                 q_value = 0.05,
                                 _temp_dir = os.path.join(tmp_dir, 'ray_spill'))

2023-01-15 07:53:26,502	INFO worker.py:1519 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(macs_call_peak_ray pid=2044428) 2023-01-15 07:53:39,567 cisTopic     INFO     Calling peaks for Stromal with /cellar/users/aklie/opt/miniconda3/envs/scenicplus/bin/macs2 callpeak --treatment mouse_adrenal/scATAC/consensus_peak_calling/pseudobulk_bed_files/Stromal.bed.gz --name Stromal  --outdir mouse_adrenal/scATAC/consensus_peak_calling/MACS/ --format BEDPE --gsize hs --qvalue 0.05 --nomodel --shift 73 --extsize 146 --keep-dup all --call-summits --nolambda
(macs_call_peak_ray pid=2044425) 2023-01-15 07:53:39,594 cisTopic     INFO     Calling peaks for Skeletal_muscle with /cellar/users/aklie/opt/miniconda3/envs/scenicplus/bin/macs2 callpeak --treatment mouse_adrenal/scATAC/consensus_peak_calling/pseudobulk_bed_files/Skeletal_muscle.bed.gz --name Skeletal_muscle  --outdir mouse_adrenal/scATAC/consensus_peak_calling/MACS/ --format BEDPE --gsize hs --qvalue 0.05 --nomodel --shift 73 --extsize 146 --keep-dup all --call-summits --nolambda
(macs_call_peak_ray pid=2044422) 2023-01-15 07:53:

In [30]:
import ray
ray.shutdown()

In [ ]:
pickle.dump(narrow_peaks_dict,
            open(os.path.join(work_dir, 'scATAC/consensus_peak_calling/MACS/narrow_peaks_dict.pkl'), 'wb'))

In [ ]:
from pycisTopic.iterative_peak_calling import *
# Other param
peak_half_width = 250
path_to_blacklist= os.path.join(work_dir, 'hg38-blacklist.v2.bed')
# Get consensus peaks
consensus_peaks=get_consensus_peaks(narrow_peaks_dict, peak_half_width, chromsizes=chromsizes, path_to_blacklist=path_to_blacklist)

In [ ]:
consensus_peaks.to_bed(
    path = os.path.join(work_dir, 'scATAC/consensus_peak_calling/consensus_regions.bed'),
    keep=True,
    compression='infer',
    chain=False)

In [ ]:
import pybiomart as pbm
dataset = pbm.Dataset(name='mmusculus_gene_ensembl',  host='http://www.ensembl.org')
annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
annot['Chromosome/scaffold name'] = annot['Chromosome/scaffold name'].to_numpy(dtype = str)
filter = annot['Chromosome/scaffold name'].str.contains('CHR|GL|JH|MT')
annot = annot[~filter]
annot['Chromosome/scaffold name'] = annot['Chromosome/scaffold name'].str.replace(r'(\b\S)', r'chr\1')
annot.columns=['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
annot = annot[annot.Transcript_type == 'protein_coding']

In [ ]:
from pycisTopic.qc import *
path_to_regions = {'10x_pbmc':os.path.join(work_dir, 'scATAC/consensus_peak_calling/consensus_regions.bed')}

metadata_bc, profile_data_dict = compute_qc_stats(
                fragments_dict = fragments_dict,
                tss_annotation = annot,
                stats=['barcode_rank_plot', 'duplicate_rate', 'insert_size_distribution', 'profile_tss', 'frip'],
                label_list = None,
                path_to_regions = path_to_regions,
                n_cpu = 1,
                valid_bc = None,
                n_frag = 100,
                n_bc = None,
                tss_flank_window = 1000,
                tss_window = 50,
                tss_minimum_signal_window = 100,
                tss_rolling_window = 10,
                remove_duplicates = True,
                _temp_dir = os.path.join(tmp_dir + 'ray_spill'))

if not os.path.exists(os.path.join(work_dir, 'scATAC/quality_control')):
    os.makedirs(os.path.join(work_dir, 'scATAC/quality_control'))

pickle.dump(metadata_bc,
            open(os.path.join(work_dir, 'scATAC/quality_control/metadata_bc.pkl'), 'wb'))

pickle.dump(profile_data_dict,
            open(os.path.join(work_dir, 'scATAC/quality_control/profile_data_dict.pkl'), 'wb'))

In [ ]:
QC_filters = {
    'Log_unique_nr_frag': [3.3 , None],
    'FRIP':               [0.45, None],
    'TSS_enrichment':     [5   , None],
    'Dupl_rate':          [None, None]

}

# Return figure to plot together with other metrics, and cells passing filters. Figure will be saved as pdf.
from pycisTopic.qc import *
FRIP_NR_FRAG_fig, FRIP_NR_FRAG_filter=plot_barcode_metrics(metadata_bc['10x_pbmc'],
                                       var_x='Log_unique_nr_frag',
                                       var_y='FRIP',
                                       min_x=QC_filters['Log_unique_nr_frag'][0],
                                       max_x=QC_filters['Log_unique_nr_frag'][1],
                                       min_y=QC_filters['FRIP'][0],
                                       max_y=QC_filters['FRIP'][1],
                                       return_cells=True,
                                       return_fig=True,
                                       plot=False)
# Return figure to plot together with other metrics, and cells passing filters
TSS_NR_FRAG_fig, TSS_NR_FRAG_filter=plot_barcode_metrics(metadata_bc['10x_pbmc'],
                                      var_x='Log_unique_nr_frag',
                                      var_y='TSS_enrichment',
                                      min_x=QC_filters['Log_unique_nr_frag'][0],
                                      max_x=QC_filters['Log_unique_nr_frag'][1],
                                      min_y=QC_filters['TSS_enrichment'][0],
                                      max_y=QC_filters['TSS_enrichment'][1],
                                      return_cells=True,
                                      return_fig=True,
                                      plot=False)
# Return figure to plot together with other metrics, but not returning cells (no filter applied for the duplication rate  per barcode)
DR_NR_FRAG_fig=plot_barcode_metrics(metadata_bc['10x_pbmc'],
                                      var_x='Log_unique_nr_frag',
                                      var_y='Dupl_rate',
                                      min_x=QC_filters['Log_unique_nr_frag'][0],
                                      max_x=QC_filters['Log_unique_nr_frag'][1],
                                      min_y=QC_filters['Dupl_rate'][0],
                                      max_y=QC_filters['Dupl_rate'][1],
                                      return_cells=False,
                                      return_fig=True,
                                      plot=False,
                                      plot_as_hexbin = True)

# Plot barcode stats in one figure
fig=plt.figure(figsize=(10,10))
plt.subplot(1, 3, 1)
img = fig2img(FRIP_NR_FRAG_fig)
plt.imshow(img)
plt.axis('off')
plt.subplot(1, 3, 2)
img = fig2img(TSS_NR_FRAG_fig)
plt.imshow(img)
plt.axis('off')
plt.subplot(1, 3, 3)
img = fig2img(DR_NR_FRAG_fig)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
bc_passing_filters = {'10x_pbmc':[]}
bc_passing_filters['10x_pbmc'] = list((set(FRIP_NR_FRAG_filter) & set(TSS_NR_FRAG_filter)))
pickle.dump(bc_passing_filters,
            open(os.path.join(work_dir, 'scATAC/quality_control/bc_passing_filters.pkl'), 'wb'))
print(f"{len(bc_passing_filters['10x_pbmc'])} barcodes passed QC stats")

## 4. pycisTopic analysis

# Scratch